<a href="https://colab.research.google.com/github/ayachiii/soturon/blob/main/concat_HRV_PLI_Q%E8%87%AA%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HRV、PLI、アンケート、kubios_HRVをつなげるプログラム
# 正規化、標準化の操作が必要な指標以外のHRV_and_PLI.csvを出力するプログラム

In [29]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re
import os

In [30]:
pd.set_option('display.max_columns', 30)

・人名_HRV.csv

・Kubios_HRV.csv

・mean_PLI_人名.csv

・questionnaire.csv

の四つをアップロードする。

In [31]:
SUBJECT_NAME = "kumakura"#ここは手動で適宜変更

FILE_HRV= f"{SUBJECT_NAME}_HRV.csv"          # kumakura_HRV.csv
FILE_KUBIOS_HRV = "kubios_HRV.csv"                      # kubios_HRV.csv (※もし kubios_HRV-2.csv なら、その名前に変更)
FILE_MEAN_PLI = f"mean_PLI_{SUBJECT_NAME}.csv"        # mean_PLI_kumakura.csv
FILE_QUESTIONNAIRE = "questionnarie.csv"              # questionnaire.csv

In [15]:

df_hrv = pd.read_csv(FILE_HRV, index_col=0)
df_kubios = pd.read_csv(FILE_KUBIOS_HRV, index_col=0)
df_pli = pd.read_csv(FILE_MEAN_PLI, index_col=0)
df_qn = pd.read_csv(FILE_QUESTIONNAIRE, index_col=0)

print("---HRV---")
print(df_hrv.head())
print("---Kubios_HRV---")
print(df_kubios.head())
print("---PLI---")
print(df_pli.head())
print("---questionnaire---")
print(df_qn.head())

---HRV---
                  bpm         ibi       sdnn       sdsd      rmssd     pnn20  \
rest        74.087015  809.858521  41.250305  20.146486  36.787621  0.619355   
boredom     76.136223  788.061162  38.553302  16.680887  31.258471  0.582822   
flow        79.897427  750.962861  43.874994  16.548756  29.365935  0.541322   
flow_ultra  79.017461  759.325843  67.135244  20.173773  34.786191  0.581921   
overload    76.508025  784.231454  37.720271  19.232687  32.628136  0.529762   

               pnn50  hr_mad        sd1        sd2            s   sd1/sd2  \
rest        0.183871    30.0  26.012686  52.268898  4271.480335  0.497670   
boredom     0.092025    22.0  22.103056  49.826426  3459.887136  0.443601   
flow        0.084022    30.0  20.764207  58.349347  3806.284253  0.355860   
flow_ultra  0.169492    58.0  24.591192  91.571813  7074.427022  0.268545   
overload    0.122024    26.0  23.071538  48.137820  3489.095291  0.479281   

            breathingrate          vlf        

In [16]:
dataframe_to_combine=[df_hrv,df_kubios,df_pli,df_qn]
df_combined=pd.concat(dataframe_to_combine,axis=1)

In [17]:
print("---四つを結合した後のファイル---")
pd.set_option('display.max_columns', None)
print(df_combined)

---四つを結合した後のファイル---
                  bpm         ibi       sdnn       sdsd      rmssd     pnn20  \
rest        74.087015  809.858521  41.250305  20.146486  36.787621  0.619355   
boredom     76.136223  788.061162  38.553302  16.680887  31.258471  0.582822   
flow        79.897427  750.962861  43.874994  16.548756  29.365935  0.541322   
flow_ultra  79.017461  759.325843  67.135244  20.173773  34.786191  0.581921   
overload    76.508025  784.231454  37.720271  19.232687  32.628136  0.529762   
practice          NaN         NaN        NaN        NaN        NaN       NaN   
ultra             NaN         NaN        NaN        NaN        NaN       NaN   

               pnn50  hr_mad        sd1        sd2            s   sd1/sd2  \
rest        0.183871    30.0  26.012686  52.268898  4271.480335  0.497670   
boredom     0.092025    22.0  22.103056  49.826426  3459.887136  0.443601   
flow        0.084022    30.0  20.764207  58.349347  3806.284253  0.355860   
flow_ultra  0.169492    58.0  2

# 今までは、四つのデータを結合する作業。
# ここから、正規化、標準化の作業

In [5]:
import pandas as pd
import numpy as np
import sys

# --- 0. 設定とファイル読み込み ---

SUBJECT_NAME = "kumakura"

# ファイル名の定義
FILE_HRV_KUMAKURA = f"{SUBJECT_NAME}_HRV.csv"
FILE_KUBIOS_HRV = "kubios_HRV.csv"
FILE_MEAN_PLI = f"mean_PLI_{SUBJECT_NAME}.csv"
FILE_QUESTIONNAIRE = "questionnarie.csv"

print("--- データの読み込み開始 ---")
try:
    df_hrv = pd.read_csv(FILE_HRV_KUMAKURA, index_col=0)
    df_kubios = pd.read_csv(FILE_KUBIOS_HRV, index_col=0)
    df_pli = pd.read_csv(FILE_MEAN_PLI, index_col=0)
    df_qn = pd.read_csv(FILE_QUESTIONNAIRE, index_col=0)
except FileNotFoundError as e:
    print(f"エラー: ファイルが見つかりません - {e.filename}")
    sys.exit()

# --- 1. 行名（フェーズ名）の修正と整理 ---

def fix_index_names(df):
    # 'flow_ultra' を 'ultra' に変更
    if 'flow_ultra' in df.index:
        df = df.rename(index={'flow_ultra': 'ultra'})

    # 重複削除: もし 'ultra' が2つ以上できてしまった場合、最後のものを残す（新しいデータを優先）
    # ※データ状況によっては 'first' にすべき場合もあります
    df = df[~df.index.duplicated(keep='last')]
    return df

# 全てのデータフレームに行名修正を適用
df_hrv = fix_index_names(df_hrv)
df_kubios = fix_index_names(df_kubios)
df_pli = fix_index_names(df_pli)
df_qn = fix_index_names(df_qn)

# --- 2. 結合処理 ---

# 横方向に結合
df_combined = pd.concat([df_hrv, df_kubios, df_pli, df_qn], axis=1)

# ▼▼▼ ここから書き換え ▼▼▼

# 1. もし 'practice' 行がまだ存在しない場合、空の行として作成しておく（必要なら）
if 'practice' not in df_combined.index:
    # 全カラムNaN（空）でpractice行を作る
    df_combined.loc['practice'] = np.nan

# 2. 並べたい順序をリストで指定（ここを自由に書き換えてください）
# ※ここに書いた順序通りにCSVの行が並びます
target_row_order = [
    'rest',       # 1行目
    'boredom',    # 2行目
    'flow',       # 3行目
    'ultra',      # 4行目
    'overload',   # 5行目
    'practice'    # 6行目
]

# 3. 実際に並べ替えを実行
# リストにある行だけを抜き出し、リストにない行（未知のフェーズ）は後ろにくっつける
existing_rows = [x for x in target_row_order if x in df_combined.index]
remaining_rows = [x for x in df_combined.index if x not in target_row_order]

final_order = existing_rows + remaining_rows
df_combined = df_combined.reindex(final_order)

print(f"最終的な行の順序: {df_combined.index.tolist()}")

# ▲▲▲ 書き換えここまで ▲▲▲


# --- 3. 計算ロジックの定義 ---

# ベースライン（rest）が存在するか確認
if 'rest' not in df_combined.index:
    print("エラー: 'rest' フェーズが見つかりません。差分計算ができません。")
    sys.exit()

base_line = df_combined.loc['rest']

# ▼▼▼ カラムグループの定義（お手本のカラム構成に基づく） ▼▼▼

# 1. HRV (HeartPy等) の基本指標
cols_hrv_base = ['lf', 'hf', 'lf/hf', 'min_lf', 'min_hf']
# 2. Kubios の基本指標
cols_kubios_base = ['kubios_lf', 'kubios_hf', 'kubios_lf/hf']
# 3. PLI の集中度指標
cols_pli_conc = ['concentration_FC3', 'concentration_FC4', 'concentration_FCz',
                 'max_concentration_FC3', 'max_concentration_FC4', 'max_concentration_FCz',
                 'concentration_mean', 'max_concentration_mean']

# 標準化・正規化の対象にする全カラム（SDNNなども含む）
cols_target_metrics = cols_hrv_base + cols_kubios_base + cols_pli_conc + ['sdnn', 'rmssd']


# --- 4. 差分 (Diff) の計算 ---

# 差分を計算したいカラムリスト
cols_to_diff = cols_hrv_base + cols_kubios_base + ['sdnn', 'rmssd'] # SDNN等のDiffも必要ならここに追加

for col in cols_to_diff:
    if col in df_combined.columns:
        diff_col = f"{col}_diff"
        # rest - current (restからの減少量) なのか current - rest (変化量) なのか
        # お手本の値を確認すると、rest(206) - flow(112) = +94 なので、「Base - Current」のようです
        df_combined[diff_col] = base_line[col] - df_combined[col]

        # rest行は強制的に0
        df_combined.loc['rest', diff_col] = 0


# --- 5. 標準化 (Standardization) ---
# ★重要: ここでお手本とのズレを解消します
# Pythonのzscore(ddof=0)ではなく、Pandasのstd(ddof=1/不偏分散)を手動で使います

# 差分カラムも含めた、標準化したい全カラムリストを作成
cols_diff_created = [f"{c}_diff" for c in cols_to_diff if f"{c}_diff" in df_combined.columns]
cols_for_calc = cols_target_metrics + cols_diff_created

for col in cols_for_calc:
    if col in df_combined.columns:
        std_col = f"{col}_standardized"

        # 平均と標準偏差（ddof=1: 不偏標準偏差）を計算
        # ※計算に 'practice' を含めるかどうかが値に大きく影響します。
        #  df_combined全体を使うことで、お手本（全行使用）に合わせます。
        mu = df_combined[col].mean()
        sigma = df_combined[col].std(ddof=1) # ここがポイント！ExcelのSTDEV.Sと同じ

        if sigma != 0:
            df_combined[std_col] = (df_combined[col] - mu) / sigma
        else:
            df_combined[std_col] = 0


# --- 6. 正規化 (Normalization) ---
# Min-Maxスケーリング

for col in cols_for_calc:
    if col in df_combined.columns:
        norm_col = f"{col}_normalized"

        min_val = df_combined[col].min()
        max_val = df_combined[col].max()

        if max_val - min_val != 0:
            df_combined[norm_col] = (df_combined[col] - min_val) / (max_val - min_val)
        else:
            df_combined[norm_col] = 0


# --- 7. カラム順序の整列 (お手本に完全準拠) ---

target_columns_order = [
    # Basic HRV
    'bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad',
    'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'vlf', 'lf', 'hf', 'lf/hf',
    'p_total', 'vlf_perc', 'lf_perc', 'hf_perc', 'lf_nu', 'hf_nu',
    'min_lf', 'min_hf',

    # Diff
    'lf_diff', 'hf_diff', 'lfhf_diff', # 存在しない場合はスキップされます

    # Standardized (Original)
    'lf_standardized', 'hf_standardized', 'lf/hf_standardized',
    'min_lf_standardized', 'min_hf_standardized',

    # Standardized (Diff)
    'lf_diff_standardized', 'hf_diff_standardized', 'lfhf_diff_standardized',

    # Normalized (Original)
    'lf_normalized', 'hf_normalized', 'lf/hf_normalized',
    'min_lf_normalized', 'min_hf_normalized',

    # Normalized (Diff)
    'lf_diff_normalized', 'hf_diff_normalized', 'lfhf_diff_normalized',

    # Kubios
    'kubios_lf', 'kubios_hf', 'kubios_lf/hf',
    'kubios_lf_standardized', 'kubios_hf_standardized', 'kubios_lf/hf_standardized',
    'kubios_lf_normalized', 'kubios_hf_normalized', 'kubios_lf/hf_normalized',

    # PLI Raw
    'FpZ', 'FC3', 'FC4', 'FCz', 'O1', 'O2', 'Fp2', 'mean_all', 'mean_3ch',

    # PLI Concentration
    'concentration_FC3', 'concentration_FC4', 'concentration_FCz',
    'max_concentration_FC3', 'max_concentration_FC4', 'max_concentration_FCz',
    'concentration_mean', 'max_concentration_mean',

    # PLI Standardized
    'concentration_FC3_standardized', 'concentration_FC4_standardized', 'concentration_FCz_standardized',
    'concentration_mean_standardized',
    'max_concentration_FC3_standardized', 'max_concentration_FC4_standardized', 'max_concentration_FCz_standardized',
    'max_concentration_mean_standardized',

    # Questionnaire
    'difficulty', 'fast', 'fulfillment', 'absorb', 'questionnaire_average'
]

# 実際に存在するカラムだけを抽出して並べ替え
final_cols = [c for c in target_columns_order if c in df_combined.columns]
# target_columns_orderに漏れていたカラムがあれば末尾に追加（念のため）
remaining_cols = [c for c in df_combined.columns if c not in final_cols]

df_final = df_combined[final_cols + remaining_cols]

# --- 8. 保存 ---

output_filename = "HRV_and_PLI_reconstructed.csv"
df_final.to_csv(output_filename)

print("--------------------------------------------------")
print(f"✅ 完了しました。ファイル名: {output_filename}")
print(f"行数: {len(df_final)}, 列数: {len(df_final.columns)}")
print("行の内訳:", df_final.index.tolist())
print("--------------------------------------------------")
# 確認用表示
pd.set_option('display.max_columns', None)
print(df_final.head())

--- データの読み込み開始 ---
最終的な行の順序: ['rest', 'boredom', 'flow', 'ultra', 'overload', 'practice']
--------------------------------------------------
✅ 完了しました。ファイル名: HRV_and_PLI_reconstructed.csv
行数: 6, 列数: 117
行の内訳: ['rest', 'boredom', 'flow', 'ultra', 'overload', 'practice']
--------------------------------------------------
                bpm         ibi       sdnn       sdsd      rmssd     pnn20  \
rest      74.087015  809.858521  41.250305  20.146486  36.787621  0.619355   
boredom   76.136223  788.061162  38.553302  16.680887  31.258471  0.582822   
flow      79.897427  750.962861  43.874994  16.548756  29.365935  0.541322   
ultra     79.017461  759.325843  67.135244  20.173773  34.786191  0.581921   
overload  76.508025  784.231454  37.720271  19.232687  32.628136  0.529762   

             pnn50  hr_mad        sd1        sd2            s   sd1/sd2  \
rest      0.183871    30.0  26.012686  52.268898  4271.480335  0.497670   
boredom   0.092025    22.0  22.103056  49.826426  3459.887136

↑ここまで!!!

In [ ]:
HRV_pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/*_*/ECG/*_HRV.csv")
HRV_pathes.sort()
HRV_pathes

In [ ]:
PLI_pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/*_*/PLIdata/mean_PLI_*.csv")
PLI_pathes.sort()
PLI_pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/PLIdata/mean_PLI_kumakura.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/10_tenshin/PLIdata/mean_PLI_tenshin.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/11_masanori/PLIdata/mean_PLI_masanori.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/12_enoki/PLIdata/mean_PLI_enoki.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/13_ito/PLIdata/mean_PLI_ito.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/14_yoshioka/PLIdata/mean_PLI_yoshioka.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/15_chizuru/PLIdata/mean_PLI_chizuru.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/PLIdata/mean_PLI_kim.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/PLIdata/mean_PLI_souma.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/PLIdata/mean_PLI_fujii.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_pr

In [ ]:
questionnarie_pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/*_*/questionnarie.csv")
questionnarie_pathes.sort()
questionnarie_pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/10_tenshin/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/11_masanori/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/12_enoki/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/13_ito/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/14_yoshioka/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/15_chizuru/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/questionnarie.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_to

In [ ]:
kubios_pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/*_*/kubios_HRV.csv")
kubios_pathes.sort()
kubios_pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/10_tenshin/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/11_masanori/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/12_enoki/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/13_ito/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/14_yoshioka/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/15_chizuru/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/kubios_HRV.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/kubios_HRV.csv',
 '/Users/miya

In [ ]:
hrv_df = pd.read_csv(f"{HRV_pathes[0]}")
hrv_df

,Unnamed: 0,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu,lf_diff,hf_diff,lfhf_diff
0,rest,74.087015,809.858521,41.250305,20.146486,36.787621,0.619355,0.183871,30.0,26.012686,52.268898,4271.480335,0.497670,0.300000,396.011131,338.306267,533.157937,0.634533,1267.475336,31.244090,26.691349,42.064561,38.820443,61.179557,0.000000,0.000000,0.000000
1,boredom,76.136223,788.061162,38.553302,16.680887,31.258471,0.582822,0.092025,22.0,22.103056,49.826426,3459.887136,0.443601,0.366667,518.685695,348.261920,297.243050,1.171640,1164.190666,44.553329,29.914509,25.532162,53.951857,46.048143,-9.955653,235.914887,-0.537107
2,flow,79.897427,750.962861,43.874994,16.548756,29.365935,0.541322,0.084022,30.0,20.764207,58.349347,3806.284253,0.355860,0.128217,971.981731,207.806086,79.141757,2.625745,1258.929574,77.206998,16.506570,6.286432,72.419463,27.580537,130.500182,454.016180,-1.991212
3,ultra,79.017461,759.325843,67.135244,20.173773,34.786191,0.581921,0.169492,58.0,24.591192,91.571813,7074.427022,0.268545,0.166667,1381.615908,275.811328,124.147392,2.221644,1781.574629,77.550269,15.481323,6.968408,68.959949,31.040051,62.494939,409.010545,-1.587111
4,overload,76.508025,784.231454,37.720271,19.232687,32.628136,0.529762,0.122024,26.0,23.071538,48.137820,3489.095291,0.479281,0.133333,521.107310,232.051713,215.334734,1.077633,968.493757,53.805954,23.960063,22.233983,51.868293,48.131707,106.254554,317.823203,-0.443100


In [ ]:
pli_df = pd.read_csv(f"{PLI_pathes[0]}")
pli_df

,Unnamed: 0,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,concentration_mean
0,rest,0.47038,0.71824,0.53059,0.78305,0.35243,0.28658,0.39018,0.504493,0.677293,NaN,NaN,NaN,NaN
1,practice,0.27293,0.38358,0.47002,0.54056,0.34591,0.32768,0.19776,0.362634,0.464720,0.209593,0.060921,0.174494,0.148336
2,boredom,0.28649,0.43996,0.43761,0.54694,0.17318,0.15961,0.23876,0.326079,0.474837,0.272161,0.092863,0.222104,0.195709
3,flow,0.26200,0.42757,0.43541,0.37519,0.66146,0.52691,0.25446,0.420429,0.412723,0.290671,0.095183,0.407868,0.264574
4,ultra,0.65670,0.75545,0.80386,0.77300,0.86585,0.81846,0.60434,0.753951,0.777437,0.164482,-0.071325,0.238026,0.110395
5,overload,0.12267,0.28812,0.42067,0.50908,0.24092,0.20799,0.15147,0.277274,0.405957,0.426403,0.112814,0.271573,0.270263


In [ ]:
questionnarie_df = pd.read_csv(f"{questionnarie_pathes[0]}")
questionnarie_df

,Unnamed: 0,difficulty,fast,fulfillment,absorb,questionnaire_average
0,rest,NaN,NaN,NaN,NaN,NaN
1,practice,\t\t\t\t,NaN,NaN,NaN,NaN
2,boredom,2,2.0,5.0,2.0,2.75
3,flow,7,7.0,7.0,7.0,7.00
4,ultra,5,7.0,6.0,7.0,6.25
5,overload,7,6.0,2.0,6.0,5.25


In [ ]:
kubios_df = pd.read_csv(f"{kubios_pathes[0]}")
kubios_df

,Unnamed: 0,difficulty,fast,fulfillment,absorb,questionnaire_average
0,rest,NaN,NaN,NaN,NaN,NaN
1,practice,\t\t\t\t,NaN,NaN,NaN,NaN
2,boredom,2,2.0,5.0,2.0,2.75
3,flow,7,7.0,7.0,7.0,7.00
4,ultra,5,7.0,6.0,7.0,6.25
5,overload,7,6.0,2.0,6.0,5.25


In [ ]:
for i in range(0, int(len(HRV_pathes))):

    pli_path = PLI_pathes[i]
    pli_df = pd.read_csv(f"{pli_path}",index_col=0)

    hrv_path = HRV_pathes[i]
    hrv_df = pd.read_csv(f"{hrv_path}",index_col=0)

    questionnarie_path = questionnarie_pathes[i]
    questionnarie_df = pd.read_csv(f"{questionnarie_path}",index_col=0)

    kubios_path = kubios_pathes[i]
    kubios_df = pd.read_csv(f"{kubios_path}",index_col=0)

    df_concatted = pd.concat([hrv_df,kubios_df], axis=1) # 横（axis=1）連結(concat)
    df_concatted = pd.concat([df_concatted,pli_df], axis=1) # 横（axis=1）連結(concat)
    df_concatted = pd.concat([df_concatted,questionnarie_df], axis=1) # 横（axis=1）連結(concat)
    df_concatted.to_csv(f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{Constants.subject_name_list[i]}/HRV_and_PLI.csv")

# ↓ sandbox
esc, enterで青と緑の切り替え
青状態のときにMでマークダウン

# [pandas参考文献](https://qiita.com/ysdyt/items/9ccca82fc5b504e7913a)

In [ ]:
hrv_df[["lf","hf","lf/hf"]]

,lf,hf,lf/hf
rest,1231.720452,138.655648,8.883305
boredom,830.045789,211.114474,3.931733
flow,457.744936,155.995954,2.934339
ultra,157.002619,55.390693,2.834458
overload,415.759892,103.952783,3.999507


### 左のヘッダーはインデックスというらしい
https://note.nkmk.me/python-pandas-read-csv-tsv/

In [ ]:
hrv_df = pd.read_csv(f"{HRV_pathes[0]}", index_col=0)
hrv_df

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu,lf_diff,hf_diff,lfhf_diff
rest,74.087015,809.858521,41.250305,20.146486,36.787621,0.619355,0.183871,30.0,26.012686,52.268898,4271.480335,0.497670,0.300000,396.011131,338.306267,533.157937,0.634533,1267.475336,31.244090,26.691349,42.064561,38.820443,61.179557,0.000000,0.000000,0.000000
boredom,76.136223,788.061162,38.553302,16.680887,31.258471,0.582822,0.092025,22.0,22.103056,49.826426,3459.887136,0.443601,0.366667,518.685695,348.261920,297.243050,1.171640,1164.190666,44.553329,29.914509,25.532162,53.951857,46.048143,-9.955653,235.914887,-0.537107
flow,79.897427,750.962861,43.874994,16.548756,29.365935,0.541322,0.084022,30.0,20.764207,58.349347,3806.284253,0.355860,0.128217,971.981731,207.806086,79.141757,2.625745,1258.929574,77.206998,16.506570,6.286432,72.419463,27.580537,130.500182,454.016180,-1.991212
ultra,79.017461,759.325843,67.135244,20.173773,34.786191,0.581921,0.169492,58.0,24.591192,91.571813,7074.427022,0.268545,0.166667,1381.615908,275.811328,124.147392,2.221644,1781.574629,77.550269,15.481323,6.968408,68.959949,31.040051,62.494939,409.010545,-1.587111
overload,76.508025,784.231454,37.720271,19.232687,32.628136,0.529762,0.122024,26.0,23.071538,48.137820,3489.095291,0.479281,0.133333,521.107310,232.051713,215.334734,1.077633,968.493757,53.805954,23.960063,22.233983,51.868293,48.131707,106.254554,317.823203,-0.443100


In [ ]:
pli_df = pd.read_csv(f"{PLI_pathes[0]}", index_col=0)
pli_df

,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,concentration_mean
rest,0.47038,0.71824,0.53059,0.78305,0.35243,0.28658,0.39018,0.504493,0.677293,NaN,NaN,NaN,NaN
practice,0.27293,0.38358,0.47002,0.54056,0.34591,0.32768,0.19776,0.362634,0.464720,0.209593,0.060921,0.174494,0.148336
boredom,0.28649,0.43996,0.43761,0.54694,0.17318,0.15961,0.23876,0.326079,0.474837,0.272161,0.092863,0.222104,0.195709
flow,0.26200,0.42757,0.43541,0.37519,0.66146,0.52691,0.25446,0.420429,0.412723,0.290671,0.095183,0.407868,0.264574
ultra,0.65670,0.75545,0.80386,0.77300,0.86585,0.81846,0.60434,0.753951,0.777437,0.164482,-0.071325,0.238026,0.110395
overload,0.12267,0.28812,0.42067,0.50908,0.24092,0.20799,0.15147,0.277274,0.405957,0.426403,0.112814,0.271573,0.270263


In [ ]:
df_concatted = pd.concat([hrv_df,pli_df], axis=1)
df_concatted

,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,...,hf_diff,lfhf_diff,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,concentration_mean
rest,74.087015,809.858521,41.250305,20.146486,36.787621,0.619355,0.183871,30.0,26.012686,52.268898,4271.480335,0.497670,0.300000,396.011131,338.306267,...,0.000000,0.000000,0.47038,0.71824,0.53059,0.78305,0.35243,0.28658,0.39018,0.504493,0.677293,NaN,NaN,NaN,NaN
boredom,76.136223,788.061162,38.553302,16.680887,31.258471,0.582822,0.092025,22.0,22.103056,49.826426,3459.887136,0.443601,0.366667,518.685695,348.261920,...,235.914887,-0.537107,0.28649,0.43996,0.43761,0.54694,0.17318,0.15961,0.23876,0.326079,0.474837,0.272161,0.092863,0.222104,0.195709
flow,79.897427,750.962861,43.874994,16.548756,29.365935,0.541322,0.084022,30.0,20.764207,58.349347,3806.284253,0.355860,0.128217,971.981731,207.806086,...,454.016180,-1.991212,0.26200,0.42757,0.43541,0.37519,0.66146,0.52691,0.25446,0.420429,0.412723,0.290671,0.095183,0.407868,0.264574
ultra,79.017461,759.325843,67.135244,20.173773,34.786191,0.581921,0.169492,58.0,24.591192,91.571813,7074.427022,0.268545,0.166667,1381.615908,275.811328,...,409.010545,-1.587111,0.65670,0.75545,0.80386,0.77300,0.86585,0.81846,0.60434,0.753951,0.777437,0.164482,-0.071325,0.238026,0.110395
overload,76.508025,784.231454,37.720271,19.232687,32.628136,0.529762,0.122024,26.0,23.071538,48.137820,3489.095291,0.479281,0.133333,521.107310,232.051713,...,317.823203,-0.443100,0.12267,0.28812,0.42067,0.50908,0.24092,0.20799,0.15147,0.277274,0.405957,0.426403,0.112814,0.271573,0.270263
practice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.27293,0.38358,0.47002,0.54056,0.34591,0.32768,0.19776,0.362634,0.464720,0.209593,0.060921,0.174494,0.148336


In [ ]:
# df_concatted.o_csv("/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv")

### [右結合とか左結合とかがわかるリンク](https://ai-inter1.com/pandas-dataframe_join/#st-toc-h-1)
- 連結はconcat
    - ただただガッちゃんこする
- 結合はmerge
    - 共通キーを指定して結合する